# Day 11: Monkey in the Middle

_What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?_

* Count the total number of times each monkey inspects items over 20 rounds
* Find the two most active monkeys
* Calculate the level of monkey business by multiplying together the number of times those two monkeys inspected items

## The monkey file language

No explicit syntax rules are given for the input file. Let's make up some.

### Example

```
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0
```

### Top-level parsing rules

Starting with the definition of a Monkey: There are a few things that are consistent which could be treated as language requirements:

* indentation
* order of the lines
* text before the colon

I'll make a `Monkey` class and initialize the attributes from the input with these rules:

* leading whitespace and blank lines don't matter
* embedded whitespace (with runs of whitespace treated as a single space) is used as a delimeter
* the definition of Monkey `N` starts with a line that begins with the letter "M" at the beginning of the line. The id of the monkey comes after the first " " and ends with the ":"
* the ids in the input are numbers but no math happens so just treat it as a string
* the `if` attribute uses a similar parsing rule: between the space and ":" is the value
* the values in the example are `true` and `false`, turn them into booleans
* For each line after the monkey id line until the next monkey id line
  * the ":" delimites the division between an attribute tag and the attribute definition
  * leading whitespace and blank lines don't matter
  * the first character (non-whitespace) selects the attribute type
  * each attribute has a corresponding class definition
  * attributes are: `starting items` ("S"), `operation` ("O"), `test` ("T"), `if` ("I")
  * the `if` attributes follow `test` in all the examples but there's only one `test` so the order can be ignored

### Reduced example

This is equivalent to the snippet earlier

```
M 0:
  S: 79, 98
  O: new = old * 19
  T: divisible by 23
    I true: throw to monkey 2
    I   false: throw to monkey 3

M is another monkey 1:
Testing: divisible by 19
S : 54, 65, 75, 74
I false: throw to monkey 0
Op: new = old + 6
I true: throw to monkey 2
```

### Attribute value parsing

**Starting items**
* list of positive integers separated by commas
* not ordered, may have repeats
* represents the worry level of the item
* each integer is used to create an item object with an autogenerated item identifier and the given worry level

**Operation**
* format is `new = t1 op t2`
* `new =` is the text "new ="
* `t1`, `t2` are each either an integer or the text "old"
* `op` is an operator: only "+" and "*" show up in the input

**Test**
* format is `condition n`
* `condition` is always the text "divisible by" in the input
* `n` is an integer

**If**
* format is `action monkey n`
* `action` is always the text "throw to"
* `monkey n` is the text "monkey " followed by an integer

In [1]:
from dataclasses import dataclass

import monkey

In [2]:
def load_data():
  monkeys = monkey.from_file('input.txt')

In [3]:
def business(part : int = 1):
  monkeys = monkey.from_file('input.txt')
  def throw_item(m: monkey.MonkeyId, i: monkey.WorryItem):
    monkeys[m].catch(i)
  #rounds = 20 if part == 1 else 10000
  rounds = 20 if part == 1 else 1000
  for round in range(0, rounds):
    print(f'Round {round + 1}')
    # TODO: This should be sorted by key.
    for id, m in monkeys.items():
      print(f'Monkey {id}')
      m.take_turn(throw_item, post_inpect_worry_reduction=(part==1))
    print()
  counts = sorted([m.get_inspection_count() for m in monkeys.values()])
  print(counts[-1] * counts[-2])

business()

Round 1
Monkey 0
0~i0l84 level 308, throw to 4
0~i1l66 level 242, throw to 4
0~i2l62 level 227, throw to 7
0~i3l69 level 253, throw to 7
0~i4l88 level 322, throw to 4
0~i5l91 level 333, throw to 7
0~i6l91 level 333, throw to 7
Monkey 1
1~i0l98 level 3201, throw to 6
1~i1l50 level 833, throw to 3
1~i2l76 level 1925, throw to 3
1~i3l99 level 3267, throw to 6
Monkey 2
2~i0l72 level 24, throw to 0
2~i1l56 level 19, throw to 0
2~i2l94 level 31, throw to 0
Monkey 3
3~i0l55 level 19, throw to 5
3~i1l88 level 30, throw to 6
3~i2l90 level 30, throw to 6
3~i3l77 level 26, throw to 5
3~i4l60 level 20, throw to 5
3~i5l67 level 23, throw to 5
1~i1l50 level 278, throw to 5
1~i2l76 level 642, throw to 6
Monkey 4
4~i0l69 level 299, throw to 7
4~i1l72 level 312, throw to 7
4~i2l63 level 273, throw to 7
4~i3l60 level 260, throw to 7
4~i4l72 level 312, throw to 7
4~i5l52 level 225, throw to 7
4~i6l63 level 273, throw to 7
4~i7l78 level 338, throw to 7
0~i0l84 level 1334, throw to 7
0~i1l66 level 1048, th

In [4]:
business(part=2)

Round 1
Monkey 0
0~i0l84 level 924, throw to 4
0~i1l66 level 726, throw to 4
0~i2l62 level 682, throw to 4
0~i3l69 level 759, throw to 7
0~i4l88 level 968, throw to 4
0~i5l91 level 1001, throw to 7
0~i6l91 level 1001, throw to 7
Monkey 1
1~i0l98 level 9604, throw to 3
1~i1l50 level 2500, throw to 6
1~i2l76 level 5776, throw to 6
1~i3l99 level 9801, throw to 6
Monkey 2
2~i0l72 level 73, throw to 0
2~i1l56 level 57, throw to 0
2~i2l94 level 95, throw to 0
Monkey 3
3~i0l55 level 57, throw to 6
3~i1l88 level 90, throw to 6
3~i2l90 level 92, throw to 5
3~i3l77 level 79, throw to 5
3~i4l60 level 62, throw to 5
3~i5l67 level 69, throw to 6
1~i0l98 level 9606, throw to 6
Monkey 4
4~i0l69 level 897, throw to 7
4~i1l72 level 936, throw to 7
4~i2l63 level 819, throw to 7
4~i3l60 level 780, throw to 7
4~i4l72 level 936, throw to 7
4~i5l52 level 676, throw to 7
4~i6l63 level 819, throw to 7
4~i7l78 level 1014, throw to 7
0~i0l84 level 12012, throw to 7
0~i1l66 level 9438, throw to 7
0~i2l62 level 8

_Exercise some monkey functions_

In [5]:
from dataclasses import dataclass

import monkey
monkey.factor(1824215048377)

[1824215048377]

In [11]:
def make_factored_op(op, t2):
    # old op old
    if t2 == 'old':
        match op:
            case '+':
                return lambda x : x.append(2) ; x
            case '*':
                def f(x):
                    x.extend(x)
                    return x
                return f
    # old op int
    else:
        t2 = int(t2)
        match op:
            case '+':
                return lambda x : monkey.factor(monkey.product(x) + t2)
            case '*':
                def f(x):
                    x.append(t2)
                    return x
                return f

for t in [('+', '10'), ('*', '3'), ('*', 'old'), ('+', '3')]:
    m = make_factored_op(*t)
    a=[1,2,4]
    print(f'{t}: {m(a)}')

('+', '10'): [2, 3, 3]
('*', '3'): [1, 2, 4, 3]
('*', 'old'): [1, 2, 4, 1, 2, 4]
('+', '3'): [11, 1]
